# Kmeans Clustering 

Libraries used in this code are

In [54]:
import wikipedia
import re
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\a\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\a\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


getting stop words from nltk

In [55]:
stop_words = set(stopwords.words('english'))

Making a function to remove non-alphabatic characters from a string and a function to remove stopwords

In [56]:
def removeNonAlphbets(word):
    text = re.sub("[^a-zA-Z]", "", word)
    re.sub(' +', '', text)
    return text

In [65]:
def removeStopWords(arr):
    return [w for w in arr if not w.lower() in stop_words]

Function to create a dictionary of words that occur in the article to base the clustering on

In [66]:
def createDictionaryOfAllWords(aritcle):
    createDictionaryOfAllWords.dictionary = {}
    arr = aritcle.split(" ")
    for i in range(len(arr)):
        arr[i] = removeNonAlphbets(arr[i])
    unique_words = set(arr)
    for i in unique_words:
        if i not in createDictionaryOfAllWords.dictionary.keys():
            createDictionaryOfAllWords.dictionary[i] = 0
    return createDictionaryOfAllWords.dictionary

Euclidean distance function to calculate the distance between two vectors

In [67]:

def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

Kmeans clustering function

In [68]:
def get_clusters(data, k):
    centroids = data.sample(n=k)
    clusters = [[] for _ in centroids.index]
    if type(data) is pd.DataFrame:
        while True:
            old_centroids = centroids.copy()
            for i in data.index:
                distances = []
                for j in centroids.index:
                    distances.append(euclidean_distance(data.loc[i], centroids.loc[j]))
                clusters[distances.index(np.min(distances))].append([x for x in data.loc[i]])
            if old_centroids.equals(centroids):
                break
            for i in range(k):
                centroids[i] = np.mean(clusters[i], axis=0)
    else:
        while True:
            old_centroids = centroids.copy()
            for i in data.index:
                distances = []
                for j in centroids.index:
                    distances.append(euclidean_distance(data[i], centroids[j]))
                clusters[distances.index(np.min(distances))].append(data[i])
            if old_centroids.equals(centroids):
                break
            for i in range(k):
                centroids[i] = np.mean(clusters[i], axis=0)

    return centroids, clusters

Function to create a dictionary of words that occur in the article to base the clustering on

In [69]:
def frequency(article):
    dictionary = {}
    arr = article.strip().split(" ")
    for i in range(len(arr)):
        arr[i] = removeNonAlphbets(arr[i])
    arr = removeStopWords(arr)
    unique_words = set(arr)
    for i in unique_words:
        if i not in dictionary.keys():
            dictionary[i] = 1
    for i in unique_words:
        dictionary[i]=arr.count(i)
    return dictionary

Reading the index file containing the article titles, getting the article content from wikipedia and creating a dictionary of words that occur in the article to base the clustering on

In [70]:
AllDict = {}
articleDictionaries = []
count = 10 #number of articles to be used
with open("D:/Wikipedia/wikipedia/enwiki-20220401-pages-articles-multistream-index.txt") as infile:
    
    articles = []
    for line in infile:
        arr = line.split(':')
        print("article{} : {}".format(count, arr[2]))
        article = wikipedia.page(arr[2]).content
        articles.append([arr[2], article])
        count -= 1
        if count <= 0:
            break
    for article in articles:
        articleDictionaries.append(frequency(article[1]))
        dictF = createDictionaryOfAllWords(article[1])
        for (i, j) in dictF.items():
            AllDict[i] = j

    for i in range(len(articleDictionaries)):
        for n, v in AllDict.items():
            if n not in articleDictionaries[i]:
                articleDictionaries[i][n] = 0

article10 : AccessibleComputing

article9 : Anarchism

article8 : AfghanistanHistory

article7 : AfghanistanGeography

article6 : AfghanistanPeople

article5 : AfghanistanCommunications

article4 : AfghanistanTransportations

article3 : AfghanistanMilitary

article2 : AfghanistanTransnationalIssues

article1 : AssistiveTechnology



Wrting the frequency of each word in the article to a csv file

In [71]:
with open("input.csv", 'w') as file:
    (k := next(iter(AllDict)), AllDict.pop(k))
    size = len(AllDict)
    for i in AllDict.keys():
        size-=1
        if size == 0:
            file.write(i)
        else:
            file.write(i + ",")
    file.write("\n")
    for itr in range(len(articleDictionaries)):
        for i, v in AllDict.items():
            file.write("{},".format(articleDictionaries[itr][i]))
        file.write("\n")

In [72]:
k = 3
data = pd.read_csv("input.csv")
data.head()

,them,present,without,so,strain,to,annual,braille,key,items,...,unlit,inhibiting,offroad,accommodate,accomplish,surgical,Nursing,Patenting,healthmonitoring,sit
0,1,2,0,1,0,1,1,5,1,2,...,0,0,0,0,0,0,0,0,0,NaN
0,0,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
0,4,1,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,NaN
0,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,NaN
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
